# 🗺️ Map Location Data Collector for Google Colab

This notebook collects location data from RapidAPI and uploads to BigQuery.

## 📋 Setup Instructions:

1. **Set up Colab Secrets** (🔑 icon in left sidebar):
   - `RAPIDAPI_KEY` - Your RapidAPI key
   - `BIGQUERY_KEY_JSON` - Your BigQuery service account JSON (as a string)

2. **Run cells in order**

---

## 📦 Step 1: Install Required Libraries

In [ ]:
!pip install -q pandas-gbq google-auth google-cloud-bigquery db-dtypes

## 📚 Step 2: Import Libraries

In [ ]:
import requests
import pandas as pd
import json
from typing import Optional, Dict, Any, List
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud import bigquery
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

## ⚙️ Step 3: Configuration

In [ ]:
# Configuration - Update these if needed
PROJECT_ID = 'shopper-reviews-477306'
DATASET_ID = 'place_data'
TABLE_ID = 'Map_location'

# Colab Secret names
RAPIDAPI_KEY_SECRET = 'RAPIDAPI_KEY'
BIGQUERY_KEY_SECRET = 'BIGQUERY_KEY_JSON'

# In-memory cache for API calls
API_CACHE: Dict[str, Any] = {}

print("✅ Configuration loaded!")
print(f"   Project: {PROJECT_ID}")
print(f"   Dataset: {DATASET_ID}")
print(f"   Table: {TABLE_ID}")

## 🔍 Step 4: Define Data Collection Functions

In [ ]:
def search_by_place_name(place_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches data for a single place from RapidAPI.
    Uses Colab Secrets for API key.
    """
    # Check cache first
    if place_name in API_CACHE:
        print(f"📦 Loading '{place_name}' from cache")
        return API_CACHE[place_name]

    print(f"🔍 Searching for '{place_name}'...")

    # Get API key from Colab Secrets
    try:
        API_KEY = userdata.get(RAPIDAPI_KEY_SECRET)
    except Exception as e:
        print(f"❌ Error: Could not get '{RAPIDAPI_KEY_SECRET}' from Colab Secrets")
        print(f"   Make sure to add it in the Secrets panel (🔑 icon)")
        return None

    API_HOST = "google-search-master-mega.p.rapidapi.com"
    url = f"https://{API_HOST}/maps"
    
    querystring = {"q": place_name, "hl": "en", "page": "1"}
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST
    }

    try:
        response = requests.get(url, headers=headers, params=querystring, timeout=10)

        if response.status_code == 200:
            data = response.json()
            API_CACHE[place_name] = data  # Cache the result
            print(f"✅ Found data for '{place_name}'")
            return data
        else:
            print(f"❌ API Error: Status code {response.status_code}")
            print(f"   Response: {response.text[:200]}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None


def collect_places_for_query(query: str) -> Optional[pd.DataFrame]:
    """
    Collects and normalizes place data for a single query.
    """
    results_data = search_by_place_name(query)

    if results_data and 'places' in results_data and results_data['places']:
        try:
            df = pd.json_normalize(results_data['places'])
            df['search_query'] = query
            print(f"✅ Collected {len(df)} places for '{query}'")
            return df
        except Exception as e:
            print(f"❌ Error processing data: {e}")
            return None
    else:
        print(f"⚠️ No places found for '{query}'")
        return None


def run_data_collection_loop() -> Optional[pd.DataFrame]:
    """
    Interactive loop to collect user queries.
    Type 'exit' to stop collecting.
    """
    all_dataframes_list: List[pd.DataFrame] = []

    print("\n" + "="*60)
    print("🗺️  PLACE SEARCHER")
    print("="*60)
    print("Type place names to search. Type 'exit' when done.\n")

    while True:
        try:
            query = input("\n🔍 Enter place name: ").strip()
        except KeyboardInterrupt:
            print("\n⏹️ Stopping data collection...")
            break

        if query.lower() == 'exit':
            print("⏹️ Exiting data collection...")
            break

        if query:
            df = collect_places_for_query(query)
            if df is not None:
                all_dataframes_list.append(df)
                print(f"📊 Total queries collected: {len(all_dataframes_list)}")
        else:
            print("⚠️ Please enter a place name")

    if not all_dataframes_list:
        print("\n⚠️ No data was collected")
        return None

    return pd.concat(all_dataframes_list, ignore_index=True)


print("✅ Data collection functions defined!")

## 📊 Step 5: COLLECT DATA (Interactive)

Run this cell and enter place names one by one. Type `exit` when done.

In [ ]:
# Collect data interactively
collected_data_df = run_data_collection_loop()

# Display results
if collected_data_df is not None and not collected_data_df.empty:
    print("\n" + "="*60)
    print("✅ DATA COLLECTION COMPLETE")
    print("="*60)
    print(f"📊 Total places collected: {len(collected_data_df)}")
    print(f"📋 Total columns: {len(collected_data_df.columns)}")
    print("\n🔍 Preview (first 5 rows):")
    display(collected_data_df.head())
    print("\n✅ Data is ready for upload!")
else:
    print("\n❌ No data to process")

## 🔄 Step 5b: ALTERNATIVE - Batch Collection (Optional)

Use this instead of Step 5 if you want to process multiple places at once without interactive input.

In [ ]:
# OPTIONAL: Batch mode - Define your places here
places_to_search = [
    "Pizza New York",
    "Sushi Tokyo",
    "Coffee Shop Paris",
    # Add more places here...
]

# Collect data
all_dataframes_list = []
for place in places_to_search:
    df = collect_places_for_query(place)
    if df is not None:
        all_dataframes_list.append(df)

if all_dataframes_list:
    collected_data_df = pd.concat(all_dataframes_list, ignore_index=True)
    print("\n✅ Batch collection complete!")
    print(f"📊 Total places collected: {len(collected_data_df)}")
    display(collected_data_df.head())
else:
    print("❌ No data collected")
    collected_data_df = None

## 🔧 Step 6: Define BigQuery Upload Functions

In [ ]:
def get_bigquery_client() -> Optional[bigquery.Client]:
    """
    Creates BigQuery client using Colab Secrets.
    """
    try:
        # Get credentials JSON from Colab Secrets
        credentials_json = userdata.get(BIGQUERY_KEY_SECRET)
        credentials_dict = json.loads(credentials_json)
        
        # Create credentials object
        credentials = service_account.Credentials.from_service_account_info(
            credentials_dict,
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
        
        # Create BigQuery client
        client = bigquery.Client(credentials=credentials, project=PROJECT_ID)
        print(f"✅ Connected to BigQuery project: {PROJECT_ID}")
        return client
        
    except Exception as e:
        print(f"❌ Error creating BigQuery client: {e}")
        print(f"   Make sure '{BIGQUERY_KEY_SECRET}' is set in Colab Secrets")
        return None


def upload_to_bigquery(df: pd.DataFrame, table_id: str = None) -> bool:
    """
    Uploads DataFrame to BigQuery.
    """
    if df is None or df.empty:
        print("⚠️ Cannot upload empty DataFrame")
        return False

    client = get_bigquery_client()
    if not client:
        return False

    table_id = table_id or f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",  # Append to existing table
        autodetect=True,  # Auto-detect schema
    )

    try:
        print(f"\n⏳ Uploading {len(df)} rows to BigQuery...")
        print(f"   Table: {table_id}")
        
        job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
        job.result()  # Wait for upload to complete

        print(f"\n✅ Successfully uploaded {len(df)} rows!")
        print(f"   View data at: https://console.cloud.google.com/bigquery?project={PROJECT_ID}")
        return True
        
    except Exception as e:
        print(f"❌ Upload error: {e}")
        return False


print("✅ BigQuery functions defined!")

## ☁️ Step 7: Upload to BigQuery

Upload the collected data to BigQuery.

In [ ]:
# Upload to BigQuery
if collected_data_df is not None and not collected_data_df.empty:
    print("="*60)
    print("☁️  UPLOADING TO BIGQUERY")
    print("="*60)
    
    success = upload_to_bigquery(collected_data_df)
    
    if success:
        print("\n🎉 Upload complete! Your data is now in BigQuery.")
    else:
        print("\n⚠️ Upload failed. Check the error messages above.")
else:
    print("❌ No data to upload. Run Step 5 first to collect data.")

## 💾 Step 8 (Optional): Save to CSV

Download the data as a CSV file instead of or in addition to uploading to BigQuery.

In [ ]:
# Save to CSV file
if collected_data_df is not None and not collected_data_df.empty:
    filename = "map_locations_data.csv"
    collected_data_df.to_csv(filename, index=False)
    print(f"✅ Data saved to: {filename}")
    print(f"📊 Rows: {len(collected_data_df)} | Columns: {len(collected_data_df.columns)}")
    
    # Download the file
    from google.colab import files
    files.download(filename)
    print(f"⬇️ Download started for {filename}")
else:
    print("❌ No data to save")

## 📊 Step 9 (Optional): View Data Summary

Get a detailed summary of the collected data.

In [ ]:
# Data summary
if collected_data_df is not None and not collected_data_df.empty:
    print("="*60)
    print("📊 DATA SUMMARY")
    print("="*60)
    
    print(f"\n📈 Shape: {collected_data_df.shape[0]} rows × {collected_data_df.shape[1]} columns")
    
    print("\n📋 Column names:")
    for i, col in enumerate(collected_data_df.columns, 1):
        print(f"   {i}. {col}")
    
    print("\n🔢 Data types:")
    print(collected_data_df.dtypes)
    
    print("\n📊 Full data preview:")
    display(collected_data_df)
else:
    print("❌ No data available")

---

## 🎉 Done!

### What you did:
1. ✅ Collected location data from RapidAPI
2. ✅ Processed and normalized the data
3. ✅ Uploaded to BigQuery (or saved as CSV)

### Next steps:
- Run Step 5 again to collect more data
- Query your data in BigQuery
- Analyze your location data!

---